# PIK3CA Mutation Trans Effect on Acetylproteomics

This notebook analyzes the trans effect of PIK3CA mutation on interacting and other proteins Acetylproteomics, in Endometrial and Colon cancer.

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import collections
import re
import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import cptac.algorithms as al

en = cptac.Endometrial()
co = cptac.Colon()

### Specify Gene

In [2]:
gene = "PIK3CA"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
#omics = "transcriptomics"
#omics = "phosphoproteomics"
omics = "acetylproteomics"

# Interacting Proteins: Acetylproteomics

## Endometrial

### Generate interacting protein list

In [4]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = en.get_acetylproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
SMARCC1
SUPT16H
CREBBP
SMARCB1
SMARCA2
SMARCC2
KMT2D
ARID1A
SMARCE1
BAZ1B
ARID1B
NF1
SMARCA4
ACTL6A
DPF2
ING1
PGR


### Test for significant comparisons in any of interacting proteins

In [5]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

NF1 did not match any columns in acetylproteomics dataframe. NF1_acetylproteomics column inserted, but filled with NaN.
Doing t-test comparisons



/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


                      Comparison       P_Value
0  SMARCC2-K599_acetylproteomics  8.624186e-07
1   ARID1A-K980_acetylproteomics  3.187381e-04





## Colon

Acetylproteomic data does not exist for the Colon dataset

# All Proteins: Phosphoproteomics

## Endometrial

In [6]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = en.get_proteomics()
    all_proteins = proteomics.columns
    all_proteins = list(all_proteins)

    all_proteins_no_dash = []
    for ap in all_proteins:
        if '-' not in ap:
            all_proteins_no_dash.append(ap)
            
    #all_proteins = all_proteins[:500]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins_no_dash)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  ARID1A
A1BG did not match any columns in acetylproteomics dataframe. A1BG_acetylproteomics column inserted, but filled with NaN.
A2ML1 did not match any columns in acetylproteomics dataframe. A2ML1_acetylproteomics column inserted, but filled with NaN.
A4GALT did not match any columns in acetylproteomics dataframe. A4GALT_acetylproteomics column inserted, but filled with NaN.
AAAS did not match any columns in acetylproteomics dataframe. AAAS_acetylproteomics column inserted, but filled with NaN.
AADAT did not match any columns in acetylproteomics dataframe. AADAT_acetylproteomics column inserted, but filled with NaN.
AAED1 did not match any columns in acetylproteomics dataframe. AAED1_acetylproteomics column inserted, but filled with NaN.
AAMDC did not match any columns in acetylproteomics dataframe. AAMDC_acetylproteomics column inserted, but filled with NaN.
AAMP did not match any columns in acetylproteomics dataframe. AAMP_acetylproteomics column inserted, but filled with Na

ADHFE1 did not match any columns in acetylproteomics dataframe. ADHFE1_acetylproteomics column inserted, but filled with NaN.
ADIPOQ did not match any columns in acetylproteomics dataframe. ADIPOQ_acetylproteomics column inserted, but filled with NaN.
ADIPOR1 did not match any columns in acetylproteomics dataframe. ADIPOR1_acetylproteomics column inserted, but filled with NaN.
ADM did not match any columns in acetylproteomics dataframe. ADM_acetylproteomics column inserted, but filled with NaN.
ADO did not match any columns in acetylproteomics dataframe. ADO_acetylproteomics column inserted, but filled with NaN.
ADPGK did not match any columns in acetylproteomics dataframe. ADPGK_acetylproteomics column inserted, but filled with NaN.
ADPRH did not match any columns in acetylproteomics dataframe. ADPRH_acetylproteomics column inserted, but filled with NaN.
ADPRM did not match any columns in acetylproteomics dataframe. ADPRM_acetylproteomics column inserted, but filled with NaN.
ADRA2A d

ANGPTL4 did not match any columns in acetylproteomics dataframe. ANGPTL4_acetylproteomics column inserted, but filled with NaN.
ANK2 did not match any columns in acetylproteomics dataframe. ANK2_acetylproteomics column inserted, but filled with NaN.
ANKHD1 did not match any columns in acetylproteomics dataframe. ANKHD1_acetylproteomics column inserted, but filled with NaN.
ANKIB1 did not match any columns in acetylproteomics dataframe. ANKIB1_acetylproteomics column inserted, but filled with NaN.
ANKLE2 did not match any columns in acetylproteomics dataframe. ANKLE2_acetylproteomics column inserted, but filled with NaN.
ANKMY2 did not match any columns in acetylproteomics dataframe. ANKMY2_acetylproteomics column inserted, but filled with NaN.
ANKRD10 did not match any columns in acetylproteomics dataframe. ANKRD10_acetylproteomics column inserted, but filled with NaN.
ANKRD11 did not match any columns in acetylproteomics dataframe. ANKRD11_acetylproteomics column inserted, but filled 

ARHGAP30 did not match any columns in acetylproteomics dataframe. ARHGAP30_acetylproteomics column inserted, but filled with NaN.
ARHGAP31 did not match any columns in acetylproteomics dataframe. ARHGAP31_acetylproteomics column inserted, but filled with NaN.
ARHGAP32 did not match any columns in acetylproteomics dataframe. ARHGAP32_acetylproteomics column inserted, but filled with NaN.
ARHGAP39 did not match any columns in acetylproteomics dataframe. ARHGAP39_acetylproteomics column inserted, but filled with NaN.
ARHGAP4 did not match any columns in acetylproteomics dataframe. ARHGAP4_acetylproteomics column inserted, but filled with NaN.
ARHGAP42 did not match any columns in acetylproteomics dataframe. ARHGAP42_acetylproteomics column inserted, but filled with NaN.
ARHGAP44 did not match any columns in acetylproteomics dataframe. ARHGAP44_acetylproteomics column inserted, but filled with NaN.
ARHGAP45 did not match any columns in acetylproteomics dataframe. ARHGAP45_acetylproteomics 

ATP2A3 did not match any columns in acetylproteomics dataframe. ATP2A3_acetylproteomics column inserted, but filled with NaN.
ATP2B2 did not match any columns in acetylproteomics dataframe. ATP2B2_acetylproteomics column inserted, but filled with NaN.
ATP2C2 did not match any columns in acetylproteomics dataframe. ATP2C2_acetylproteomics column inserted, but filled with NaN.
ATP5MC1 did not match any columns in acetylproteomics dataframe. ATP5MC1_acetylproteomics column inserted, but filled with NaN.
ATP5MD did not match any columns in acetylproteomics dataframe. ATP5MD_acetylproteomics column inserted, but filled with NaN.
ATP5MF did not match any columns in acetylproteomics dataframe. ATP5MF_acetylproteomics column inserted, but filled with NaN.
ATP6AP1 did not match any columns in acetylproteomics dataframe. ATP6AP1_acetylproteomics column inserted, but filled with NaN.
ATP6AP2 did not match any columns in acetylproteomics dataframe. ATP6AP2_acetylproteomics column inserted, but fil

...

UBAP2 did not match any columns in acetylproteomics dataframe. UBAP2_acetylproteomics column inserted, but filled with NaN.
UBB did not match any columns in acetylproteomics dataframe. UBB_acetylproteomics column inserted, but filled with NaN.
UBE2B did not match any columns in acetylproteomics dataframe. UBE2B_acetylproteomics column inserted, but filled with NaN.
UBE2C did not match any columns in acetylproteomics dataframe. UBE2C_acetylproteomics column inserted, but filled with NaN.
UBE2D1 did not match any columns in acetylproteomics dataframe. UBE2D1_acetylproteomics column inserted, but filled with NaN.
UBE2D4 did not match any columns in acetylproteomics dataframe. UBE2D4_acetylproteomics column inserted, but filled with NaN.
UBE2F did not match any columns in acetylproteomics dataframe. UBE2F_acetylproteomics column inserted, but filled with NaN.
UBE2G1 did not match any columns in acetylproteomics dataframe. UBE2G1_acetylproteomics column inserted, but filled with NaN.
UBE2G2

UTP14A did not match any columns in acetylproteomics dataframe. UTP14A_acetylproteomics column inserted, but filled with NaN.
UTP15 did not match any columns in acetylproteomics dataframe. UTP15_acetylproteomics column inserted, but filled with NaN.
UTP20 did not match any columns in acetylproteomics dataframe. UTP20_acetylproteomics column inserted, but filled with NaN.
UTP23 did not match any columns in acetylproteomics dataframe. UTP23_acetylproteomics column inserted, but filled with NaN.
UTP25 did not match any columns in acetylproteomics dataframe. UTP25_acetylproteomics column inserted, but filled with NaN.
UTP3 did not match any columns in acetylproteomics dataframe. UTP3_acetylproteomics column inserted, but filled with NaN.
UTP6 did not match any columns in acetylproteomics dataframe. UTP6_acetylproteomics column inserted, but filled with NaN.
UVRAG did not match any columns in acetylproteomics dataframe. UVRAG_acetylproteomics column inserted, but filled with NaN.
UVSSA did 

WDR81 did not match any columns in acetylproteomics dataframe. WDR81_acetylproteomics column inserted, but filled with NaN.
WDR82 did not match any columns in acetylproteomics dataframe. WDR82_acetylproteomics column inserted, but filled with NaN.
WDR83 did not match any columns in acetylproteomics dataframe. WDR83_acetylproteomics column inserted, but filled with NaN.
WDR89 did not match any columns in acetylproteomics dataframe. WDR89_acetylproteomics column inserted, but filled with NaN.
WDR90 did not match any columns in acetylproteomics dataframe. WDR90_acetylproteomics column inserted, but filled with NaN.
WDR92 did not match any columns in acetylproteomics dataframe. WDR92_acetylproteomics column inserted, but filled with NaN.
WDSUB1 did not match any columns in acetylproteomics dataframe. WDSUB1_acetylproteomics column inserted, but filled with NaN.
WDTC1 did not match any columns in acetylproteomics dataframe. WDTC1_acetylproteomics column inserted, but filled with NaN.
WDYHV1

ZCRB1 did not match any columns in acetylproteomics dataframe. ZCRB1_acetylproteomics column inserted, but filled with NaN.
ZDHHC13 did not match any columns in acetylproteomics dataframe. ZDHHC13_acetylproteomics column inserted, but filled with NaN.
ZDHHC17 did not match any columns in acetylproteomics dataframe. ZDHHC17_acetylproteomics column inserted, but filled with NaN.
ZDHHC18 did not match any columns in acetylproteomics dataframe. ZDHHC18_acetylproteomics column inserted, but filled with NaN.
ZDHHC2 did not match any columns in acetylproteomics dataframe. ZDHHC2_acetylproteomics column inserted, but filled with NaN.
ZDHHC20 did not match any columns in acetylproteomics dataframe. ZDHHC20_acetylproteomics column inserted, but filled with NaN.
ZDHHC21 did not match any columns in acetylproteomics dataframe. ZDHHC21_acetylproteomics column inserted, but filled with NaN.
ZDHHC3 did not match any columns in acetylproteomics dataframe. ZDHHC3_acetylproteomics column inserted, but f

ZNF548 did not match any columns in acetylproteomics dataframe. ZNF548_acetylproteomics column inserted, but filled with NaN.
ZNF551 did not match any columns in acetylproteomics dataframe. ZNF551_acetylproteomics column inserted, but filled with NaN.
ZNF552 did not match any columns in acetylproteomics dataframe. ZNF552_acetylproteomics column inserted, but filled with NaN.
ZNF561 did not match any columns in acetylproteomics dataframe. ZNF561_acetylproteomics column inserted, but filled with NaN.
ZNF562 did not match any columns in acetylproteomics dataframe. ZNF562_acetylproteomics column inserted, but filled with NaN.
ZNF565 did not match any columns in acetylproteomics dataframe. ZNF565_acetylproteomics column inserted, but filled with NaN.
ZNF568 did not match any columns in acetylproteomics dataframe. ZNF568_acetylproteomics column inserted, but filled with NaN.
ZNF570 did not match any columns in acetylproteomics dataframe. ZNF570_acetylproteomics column inserted, but filled wi

## Colon

Colon Dataset doesn't contain acetylproteomic data